# Week 3: Identify Risk Factors for Infection

<span style="color:red">
**UPDATE**

Thank you again for the previous analysis. We will next be publishing a public health advisory that warns of specific infection risk factors of which individuals should be aware. Please advise as to which population characteristics are associated with higher infection rates. 
</span>

Your goal for this notebook will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [18]:
import cudf
import cuml
import pandas as pd

## Load Data

Begin by loading the data you've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook you will need all columns of the data.

In [2]:
gdf = cudf.read_csv("./data/week3.csv")
gdf.dtypes

age             int64
sex            object
employment     object
infected      float64
dtype: object

In [4]:
gdf.head()

,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


In [27]:
edf = gdf.groupby(['employment']).count()
sedf = edf['infected'].sort_values(ascending = False)
print(sedf)

U          12459115
V          10098466
Z           7161907
Q           3802602
G           3549465
P           3006149
C           2653753
M           2214336
F           2075628
O           1843446
R, S, T     1669197
I           1556575
H           1398342
N           1367137
J           1180372
K           1122406
B, D, E      486785
L            346470
A            305755
X            181988
Name: infected, dtype: int32


In [54]:
pd_sedf = pd.DataFrame(sedf.to_pandas())
pd_sedf['net_infected'] = pd_sedf['infected']
pd_sedf = pd_sedf.drop(['infected'], axis =1)
pd_sedf

,net_infected
U,12459115
V,10098466
Z,7161907
Q,3802602
G,3549465
P,3006149
C,2653753
M,2214336
F,2075628
O,1843446


## Calculate Infection Rates by Employment Code

Convert the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [55]:
gdf['infected'] = gdf['infected'].astype('float32')

Now, produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lower** than for type "X", even though more people within that employment type were infected.

In [56]:
rdf = gdf[gdf['infected']== 1.0].groupby(['employment']).count()
srdf = rdf['infected'].sort_values(ascending = False)
print(srdf)

V          76648
Q          48505
Z          40498
P          18609
G          17561
I          16116
M          10578
C          10301
O           9741
R, S, T     8997
F           6604
N           6541
K           5356
H           4737
J           4650
U           2702
B, D, E     1837
L           1722
A           1178
X            826
Name: infected, dtype: int32


In [57]:
pd_srdf = pd.DataFrame(srdf.to_pandas())
pd_srdf

,infected
V,76648
Q,48505
Z,40498
P,18609
G,17561
I,16116
M,10578
C,10301
O,9741
"R, S, T",8997


In [60]:
ndf = pd.concat([pd_srdf,pd_sedf], axis=1)
ndf['rate'] = ndf['infected']/ndf['net_infected']*100

In [63]:
ndf

,infected,net_infected,rate
V,76648,10098466,0.759006
Q,48505,3802602,1.275574
Z,40498,7161907,0.565464
P,18609,3006149,0.619031
G,17561,3549465,0.494751
I,16116,1556575,1.035350
M,10578,2214336,0.477705
C,10301,2653753,0.388167
O,9741,1843446,0.528413
"R, S, T",8997,1669197,0.539002


In [62]:
sndf = ndf['rate'].sort_values(ascending = False)
sndf

Q          1.275574
I          1.035350
V          0.759006
P          0.619031
Z          0.565464
R, S, T    0.539002
O          0.528413
L          0.497013
G          0.494751
N          0.478445
M          0.477705
K          0.477189
X          0.453876
J          0.393944
C          0.388167
A          0.385276
B, D, E    0.377374
H          0.338758
F          0.318169
U          0.021687
Name: rate, dtype: float64

Finally, read in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.

In [10]:
edf = cudf.read_csv("./data/code_guide.csv")
edf

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"
5,H,Transport & storage
6,I,Accommodation & food services
7,J,Information & communication
8,K,Financial & insurance activities
9,L,Real estate activities


## Calculate Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Group by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [17]:
rdf = gdf[gdf['infected']== 1.0].groupby(['employment', 'sex']).count()
srdf = rdf['infected'].sort_values(ascending = False)
print(srdf)

employment  sex
V           f      59713
Q           f      44164
Z           f      33430
V           m      16935
P           f      16453
G           f      12715
I           f      12344
O           f       7536
M           f       7456
R, S, T     f       7075
Z           m       7068
C           f       5251
            m       5050
G           m       4846
N           f       4678
F           m       4587
Q           m       4341
K           f       3792
I           m       3772
M           m       3122
H           m       2830
J           f       2466
O           m       2205
J           m       2184
P           m       2156
F           f       2017
U           f       2001
R, S, T     m       1922
H           f       1907
N           m       1863
K           m       1564
L           f       1381
B, D, E     f        924
            m        913
U           m        701
A           f        616
X           f        571
A           m        562
L           m        341
X        

## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 3 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to continue work in other notebooks, please shutdown the kernel.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)